In [1]:
#Used tools
import numpy as np
import pandas as pd


In [2]:
# Implementation of Logistic Regression
class LRegression():
    #Attributes
    _features = None
    _targets = None
    _weights = None
    featt = None
    tarr = None
    
    def __init__(self, feat,target):
        #TARGET IS EXPECTED TO BE changed to 1d array of 1 and zeros
        feat = np.insert(feat,0,1,axis=1) ##Adding x^0
        self._features = feat
        self._weights = np.zeros(self._features[0].size)
        self._targets = target
        self.tarr = self._targets.flatten()
        self.featt = self._features

    def fit(self, tData, corVec, lRate, itera): 
        for x in range(itera):
            N = tData.shape[0]
            grad = np.dot((corVec-self.sig(np.dot(tData,self._weights))),tData) 
            grad *= lRate
            grad /= N
            self._weights = np.add(self._weights,grad)


    def predict(self,tSet):
        return self._classify(self.sig(np.array(np.dot(tSet, self._weights),dtype=np.float32)))
    
    def cost(self,trueV,predV):
        obs = trueV.shape[0]
        
        return ((-trueV*np.log(predV))-((1-trueV)*np.log(1-predV))).sum()/obs
    def sig(self,r):
        return 1.0/(1.0+np.exp(-(np.array(r,dtype=np.float32))))
    
    def printW(self):
        return self._weights
    def _classify(self,pred):
        l = []
        for x in pred:
            if x >= 0.5:
                l.append(1)
            else:
                l.append(0)
        return np.array(l)
        
    def Accu_eval(self,pL,tL):
        n=tL.size
        return np.mean(pL == tL)

In [3]:
# Implementation of LDA
class LDA():
    _feat = None
    _targ = None
    _N = None
    _p_one = None
    _p_zero = None
    _u_one = None
    _u_zero = None
    _E_one = None
    _E_zero = None
    _E = None
    def __init__(self,features,targets):
        self._feat = features
        self._targ = targets
        self._N = targets.size #Expecting 1d Array of labels
        self._n_one = targets.sum()
        self._n_zero = (self._N-self._targ.sum())
        self._p_one = self._n_one/self._N
        self._p_zero = self._n_zero/self._N
        self._u_zero = 0
        self._u_one = 0
        for x in range(self._N): ##INDICATOR
            if self._targ[x] == 1:
                self._u_one += self._feat[x]
            else:
                self._u_zero += self._feat[x]
        self._u_zero = self._u_zero/self._n_zero #Expecting 1/0 labels passed on
        self._u_one = self._u_one/self._n_one
        self._E_one =0
        self._E_zero =0
        for x in range(self._N):
            if self._targ[x] == 1:
                self._E_one += np.outer((self._feat[x] - self._u_one),(self._feat[x] - self._u_one))/(self._N -2)
            else:
                self._E_zero += np.outer((self._feat[x] - self._u_zero),(self._feat[x] - self._u_zero))/(self._N -2)
        self._E = self._E_one + self._E_zero

    def predict(self,features):
        predResult = []
        a = np.log(self._p_one/self._p_zero)
        b = (np.dot(np.dot(self._u_one.T,np.linalg.inv(np.array(self._E,dtype=np.float32))),self._u_one)/2)
        c = (np.dot(np.dot(self._u_zero.T,np.linalg.inv(np.array(self._E,dtype=np.float32))),self._u_zero)/2)
        for x in range(features.shape[0]):
            d = np.dot(np.dot(features[x].T,np.linalg.inv(np.array(self._E,dtype=np.float32))),(self._u_one-self._u_zero))

            res = a-b+c+d
            if res > 0:
                predResult.append(1)
            else:
                predResult.append(0)
            
        return np.array(predResult)
    
    def Accu_eval(self,pL,tL):
        n=tL.size
        return np.mean(pL == tL)

In [4]:
#Implementation of 10-fold
#Returns a (kfolds,m,n) mxn is the 2d data array
class k_fold():
    _foldSize = None
    _returnArr = None
    def __init__(self,data,fold=10):
        self._returnArr = []
        #Expecting 2d np array
        self._foldSize = np.floor(data.shape[0]/fold)
        np.random.shuffle(data)
        for x in range(fold-1): #Last fold will have an extra element
            self._returnArr.append(data[int(self._foldSize*x):int(self._foldSize*(x+1)),:])
        self._returnArr.append(data[int(self._foldSize*(fold-1)):int(self._foldSize*((fold-1)+1)),:])
    def returnFolds(self):
        return np.array(self._returnArr)        

In [5]:
def applyToLR(kfold,lr): 
    print("Learning rate set to: "+str(lr))
    models = []
    for k in range(10): # Running accuracy on each fold, with the rest combined to apply
        train = None
        train = np.delete(kfold,k,0)
        newT = []
        for x in train:
            for y in x:
                newT.append(y)
        train = np.array(newT)
        test = kfold[k]
        tr_x = train[:,:-1] #Expecting last column to be Y
        tr_y = train[:,-1:].flatten()
        te_x = test[:,:-1] #Expecting last column to be Y
        te_y = test[:,-1:].flatten()
        x = LRegression(tr_x,tr_y)
        x.fit(x.featt,x.tarr,lr,1000)
        pred = x.predict(np.insert(te_x,0,1,axis=1))
        acc = x.Accu_eval(pred,te_y)
        models.append(x)
        print("At k="+str(k)+", accurcy = "+str(acc)) 
    return models
df = np.array(pd.read_csv('LReg/data/u/sonar.all-data', sep=',',header=None))
dq = np.array(pd.read_csv('LReg/data/u/parkinsons.data', sep=',',header=0))
dqq = np.delete(dq,0,1)
dqq = np.delete(dqq,16,1)
dqx = dq[:,17:18]
dp = np.append(dqq,dqx,axis=1)

print("Logistic Regression applied to Parkinsons dataset")
y = k_fold(dp)
y_models = applyToLR(y.returnFolds(),0.3)
print("Logistic Regression applied to Sonar dataset")
x = k_fold(df)
x_models = applyToLR(x.returnFolds(),0.3)


Logistic Regression applied to Parkinsons dataset
[[119.992 157.302 74.997 ... 2.3014419999999998 0.284654 1]
 [122.4 148.65 113.819 ... 2.486855 0.368674 1]
 [116.682 131.111 111.555 ... 2.342259 0.33263400000000004 1]
 ...
 [174.688 240.005 74.287 ... 2.6797720000000003 0.13172799999999998 0]
 [198.764 396.961 74.904 ... 2.138608 0.123306 0]
 [214.289 260.277 77.973 ... 2.555477 0.148569 0]] [] None
(10, 19, 23)
Learning rate set to: 0.3
At k=0, accurcy = 0.5263157894736842
At k=1, accurcy = 0.7368421052631579
At k=2, accurcy = 0.7894736842105263
At k=3, accurcy = 0.631578947368421
At k=4, accurcy = 0.5263157894736842
At k=5, accurcy = 0.3684210526315789
At k=6, accurcy = 0.15789473684210525
At k=7, accurcy = 0.8947368421052632
At k=8, accurcy = 0.5789473684210527
At k=9, accurcy = 0.8421052631578947
Logistic Regression applied to Sonar dataset
[[0.02   0.0371 0.0428 ... 0.009  0.0032 1.    ]
 [0.0453 0.0523 0.0843 ... 0.0052 0.0044 1.    ]
 [0.0262 0.0582 0.1099 ... 0.0095 0.0078 1.

In [6]:
def applyToLDA(kfold):
    models = []
    for k in range(10): # Running accuracy on each fold, with the rest combined to apply
        train = None
        train = np.delete(kfold,k,0)
        newT = []
        for x in train:
            for y in x:
                newT.append(y)
        train = np.array(newT)
        test = kfold[k]
        tr_x = train[:,:-1] #Expecting last column to be Y
        tr_y = train[:,-1:].flatten()
        te_x = test[:,:-1] #Expecting last column to be Y
        te_y = test[:,-1:].flatten()
        
        trainedModel = LDA(tr_x,tr_y)
        predictedTargets = trainedModel.predict(te_x)
        accuracy = trainedModel.Accu_eval(predictedTargets, te_y)
        models.append(x)
        print("At k="+str(k)+", accurcy = "+str(accuracy)) 
    return models

df = np.array(pd.read_csv('LReg/data/u/sonar.all-data', sep=',',header=None))
dq = np.array(pd.read_csv('LReg/data/u/parkinsons.data', sep=',',header=0))
dqq = np.delete(dq,0,1)
dqq = np.delete(dqq,16,1)
dqx = dq[:,17:18]
dp = np.append(dqq,dqx,axis=1)


print("LDA applied to Sonar dataset")
x = k_fold(df)
x_models = applyToLDA(x.returnFolds())
print("LDA applied to Parkinsons dataset")
y = k_fold(dp)
y_models = applyToLDA(y.returnFolds())

LDA applied to Sonar dataset
[[0.02   0.0371 0.0428 ... 0.009  0.0032 1.    ]
 [0.0453 0.0523 0.0843 ... 0.0052 0.0044 1.    ]
 [0.0262 0.0582 0.1099 ... 0.0095 0.0078 1.    ]
 ...
 [0.0522 0.0437 0.018  ... 0.0077 0.0031 0.    ]
 [0.0303 0.0353 0.049  ... 0.0036 0.0048 0.    ]
 [0.026  0.0363 0.0136 ... 0.0061 0.0115 0.    ]] [] None
(180, 61)
At k=0, accurcy = 0.8
(180, 61)
At k=1, accurcy = 0.8
(180, 61)
At k=2, accurcy = 0.8
(180, 61)
At k=3, accurcy = 0.9
(180, 61)
At k=4, accurcy = 0.65
(180, 61)
At k=5, accurcy = 0.65
(180, 61)
At k=6, accurcy = 0.8
(180, 61)
At k=7, accurcy = 0.8
(180, 61)
At k=8, accurcy = 0.7
(180, 61)
At k=9, accurcy = 0.85
LDA applied to Parkinsons dataset
[[119.992 157.302 74.997 ... 2.3014419999999998 0.284654 1]
 [122.4 148.65 113.819 ... 2.486855 0.368674 1]
 [116.682 131.111 111.555 ... 2.342259 0.33263400000000004 1]
 ...
 [174.688 240.005 74.287 ... 2.6797720000000003 0.13172799999999998 0]
 [198.764 396.961 74.904 ... 2.138608 0.123306 0]
 [214.289 